# MIT-GSL Uruguay 

## January 2020

---

# Instacart grocery dataset

### Source: https://www.instacart.com/datasets/grocery-shopping-2017

---

## Data preparation code
### NOT to be run in class

In [3]:
# =========================================================
# import python modules
# =========================================================

# data manipulation
import pandas as pd
import numpy as np

# text processing
from nltk import sent_tokenize, word_tokenize
from nltk.tokenize import ToktokTokenizer
import string
import re # regular expressions
from sklearn.feature_extraction.text import CountVectorizer

# visualization
%matplotlib inline
from ggplot import *

# set working directory
import os
os.chdir("/pool001/madhavk/gsl-uruguay/W-02-NLP/")

In [4]:
# =========================================================
# notebook options
# =========================================================

# pandas viewing options
pd.set_option('display.max_rows', 1000)

In [5]:
# =========================================================
# data directory for instacart data
# =========================================================

data_dir = "data/in-grocery/instacart_2017_05_01/"
os.listdir(data_dir)

['._products.csv',
 'aisles.csv',
 '._departments.csv',
 '._aisles.csv',
 'departments.csv',
 '._order_products__train.csv',
 'order_products__prior.csv',
 '._order_products__prior.csv',
 'orders.csv',
 'products.csv',
 'order_products__train.csv',
 '._orders.csv']

In [6]:
# =========================================================
# directory to store processed data
# =========================================================

process_dir = "nlp-data/in-grocery/prepared-data/"

if not os.path.exists(process_dir):
    os.makedirs(process_dir)

---

# 1. Product info data

## 1.1. Products file

In [7]:
# import data
products = pd.read_csv(data_dir + "products.csv")
print(products.shape)

(49688, 4)


In [8]:
display(products.head())

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


## 1.2. Department file

In [9]:
dept = pd.read_csv(data_dir + "departments.csv")
print(dept.shape)

(21, 2)


In [10]:
display(dept.head())

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


## 1.3. Aisle file

In [11]:
aisle = pd.read_csv(data_dir + "aisles.csv")
print(aisle.shape)

(134, 2)


In [12]:
display(aisle.head())

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


## 1.4. Merge all files

In [13]:
# merge products and aisle
products = pd.merge(products, aisle, on = "aisle_id", how = "left")
print(products.shape)

(49688, 5)


In [14]:
display(products.head())

,product_id,product_name,aisle_id,department_id,aisle
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes
1,2,All-Seasons Salt,104,13,spices seasonings
2,3,Robust Golden Unsweetened Oolong Tea,94,7,tea
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,frozen meals
4,5,Green Chile Anytime Sauce,5,13,marinades meat preparation


In [15]:
# merge products and department
products = pd.merge(products, dept, on = "department_id", how = "left")
display(products.shape)

(49688, 6)

In [16]:
display(products.head())

,product_id,product_name,aisle_id,department_id,aisle,department
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks
1,2,All-Seasons Salt,104,13,spices seasonings,pantry
2,3,Robust Golden Unsweetened Oolong Tea,94,7,tea,beverages
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,frozen meals,frozen
4,5,Green Chile Anytime Sauce,5,13,marinades meat preparation,pantry


In [17]:
# some manual checks
products["department"].value_counts()

personal care      6563
snacks             6264
pantry             5371
beverages          4365
frozen             4007
dairy eggs         3449
household          3085
canned goods       2092
dry goods pasta    1858
produce            1684
bakery             1516
deli               1322
missing            1258
international      1139
breakfast          1115
babies             1081
alcohol            1054
pets                972
meat seafood        907
other               548
bulk                 38
Name: department, dtype: int64

In [18]:
# some manual checks
products["aisle"].value_counts()

missing                          1258
candy chocolate                  1246
ice cream ice                    1091
vitamins supplements             1038
yogurt                           1026
chips pretzels                    989
tea                               894
packaged cheese                   891
frozen meals                      880
cookies cakes                     874
energy granola bars               832
hair care                         816
spices seasonings                 797
juice nectars                     792
crackers                          747
soup broth bouillon               737
baby food formula                 718
coffee                            680
refrigerated                      675
cleaning products                 655
baking ingredients                623
packaged vegetables fruits        615
asian foods                       605
nuts seeds dried fruit            582
fresh vegetables                  569
oral hygiene                      565
salad dressi

## 1.5. Save to disk

In [19]:
# save
products.to_csv(process_dir + "products-merged-v1.csv", index = False)

----

# 2. Orders data

In [20]:
# orders data
orders = pd.read_csv(data_dir + "orders.csv")
print(orders.shape)

(3421083, 7)


In [21]:
display(orders.head(200))

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0
6,550135,1,prior,7,1,9,20.0
7,3108588,1,prior,8,1,14,14.0
8,2295261,1,prior,9,1,16,0.0
9,2550362,1,prior,10,4,8,30.0


## 2.1. Split into train-val-test

In [22]:
# =========================================================
# remove test rows, re-assign train to prior and 
# then split, train-test-val
# =========================================================
print(orders.shape)
orders = orders.loc[orders["eval_set"] != "test", :]
print(orders.shape)

(3421083, 7)
(3346083, 7)


In [23]:
# aggregate to user level
users = orders.groupby("user_id").agg({"order_number": "max"})
users = users.reset_index()
users.head()

,user_id,order_number
0,1,11
1,2,15
2,3,12
3,4,5
4,5,5


In [24]:
print(users.shape)

(206209, 2)


In [25]:
# =========================================================
# split train-val-test
# =========================================================

# training set: 60%
# validation set: 20%
# test set: 20%
users["eval"] = np.random.choice(["train", "test", "val"], size = users.shape[0], p = [0.6, 0.2, 0.2])
users["eval"].value_counts()

train    123830
val       41345
test      41034
Name: eval, dtype: int64

In [26]:
display(users.head(20))

,user_id,order_number,eval
0,1,11,train
1,2,15,val
2,3,12,test
3,4,5,train
4,5,5,test
5,6,3,test
6,7,21,train
7,8,4,train
8,9,4,train
9,10,6,val


In [27]:
# =========================================================
# merge with orders data
# =========================================================
orders = pd.merge(orders, users, on = ["user_id", "order_number"], how = "left")
print(orders.shape)

(3346083, 8)


In [28]:
display(orders.head(200))

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval
0,2539329,1,prior,1,2,8,NaN,NaN
1,2398795,1,prior,2,3,7,15.0,NaN
2,473747,1,prior,3,3,12,21.0,NaN
3,2254736,1,prior,4,4,7,29.0,NaN
4,431534,1,prior,5,4,15,28.0,NaN
5,3367565,1,prior,6,2,7,19.0,NaN
6,550135,1,prior,7,1,9,20.0,NaN
7,3108588,1,prior,8,1,14,14.0,NaN
8,2295261,1,prior,9,1,16,0.0,NaN
9,2550362,1,prior,10,4,8,30.0,NaN


In [29]:
# set missing eval to prior and delete eval_set column
orders.loc[pd.isnull(orders["eval"]), "eval"] = "prior"
orders["eval"].value_counts()

prior    3139874
train     123830
val        41345
test       41034
Name: eval, dtype: int64

In [30]:
# delete eval_set column
orders.drop(labels = "eval_set", axis = 1, inplace = True)
display(orders.head())

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval
0,2539329,1,1,2,8,NaN,prior
1,2398795,1,2,3,7,15.0,prior
2,473747,1,3,3,12,21.0,prior
3,2254736,1,4,4,7,29.0,prior
4,431534,1,5,4,15,28.0,prior


## 2.2. Save to disk

In [31]:
# save orders
orders.to_csv(process_dir + "orders-split-v1.csv", index = False)

----

# 3. Prior orders

In [32]:
# prior orders
prior = pd.read_csv(data_dir + "order_products__prior.csv")
print(prior.shape)

(32434489, 4)


In [33]:
display(prior.head())

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


## 3.1. Remove in-frequent products

In [34]:
# product frequency to check top-products
prod_freq = prior["product_id"].value_counts()
print(prod_freq.shape)

(49677,)


In [35]:
display(prod_freq)

24852    472565
13176    379450
21137    264683
21903    241921
47209    213584
47766    176815
47626    152657
16797    142951
26209    140627
27845    137905
27966    137057
22935    113426
24964    109778
45007    104823
39275    100060
49683     97315
28204     89632
5876      87746
8277      85020
40706     84255
4920      82689
30391     80392
45066     79769
42265     76896
49235     76360
44632     75886
19057     75165
4605      73167
37646     72846
21616     72829
17794     72736
27104     71584
30489     71314
31717     69524
27086     69217
44359     68091
28985     67449
46979     67283
8518      66986
41950     64289
26604     61746
5077      60816
34126     60621
22035     59676
39877     58779
35951     57895
43352     57640
10749     57485
19660     56087
9076      55946
21938     55484
43961     55371
24184     55280
34969     54409
46667     52087
48679     51939
25890     50281
31506     50255
12341     50221
39928     50141
24838     49569
5450      49021
22825   

In [36]:
# keep products bought at least 200 times
min_freq = 200
prod_freq = prod_freq[prod_freq >= min_freq]
print(prod_freq.shape)

(14340,)


In [37]:
# subset prior data
prior = prior.loc[prior["product_id"].isin(list(prod_freq.index)), :]
print(prior.shape)

(30720465, 4)


In [38]:
# number of unique products
len(prior["product_id"].unique())

14340

In [39]:
display(prior.head())

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


## 3.2. Aggregate to order level from order-product level

In [40]:
# =========================================================
# aggregate to order-level to make sentences
# =========================================================


# =================================================
# roll-up product-id
# =================================================

# function to aggregate
f = {"product_id": lambda g: " ".join(g),
    "add_to_cart_order": ["count"]}

In [41]:
# format product-ids to string
prior["product_id"] = prior["product_id"].astype(str)

In [42]:
# roll-up
prior_orders = prior.groupby(["order_id"]).agg(f)
print(prior_orders.shape)

(3201694, 2)


In [43]:
display(prior_orders.head())

,product_id,add_to_cart_order
,<lambda>,count
order_id,,
2,33120 28985 9327 45918 30035 17794 40141 1819,8
3,33754 24838 17704 21903 17668 46667 17461 32665,8
4,46842 39758 27761 10054 21351 22598 34862 4028...,12
5,13176 47329 27966 23909 48370 13245 27360 6348...,22
6,40462,1


In [44]:
# =================================================
# reset column levels
# =================================================
prior_orders.columns = prior_orders.columns.droplevel(1)
prior_orders.reset_index(inplace = True)
print(prior_orders.shape)

(3201694, 3)


In [45]:
display(prior_orders.head())

,order_id,product_id,add_to_cart_order
0,2,33120 28985 9327 45918 30035 17794 40141 1819,8
1,3,33754 24838 17704 21903 17668 46667 17461 32665,8
2,4,46842 39758 27761 10054 21351 22598 34862 4028...,12
3,5,13176 47329 27966 23909 48370 13245 27360 6348...,22
4,6,40462,1


In [46]:
# rename columns
prior_orders.rename(columns = {"add_to_cart_order" : "num_products"}, inplace = True)
display(prior_orders.head())

,order_id,product_id,num_products
0,2,33120 28985 9327 45918 30035 17794 40141 1819,8
1,3,33754 24838 17704 21903 17668 46667 17461 32665,8
2,4,46842 39758 27761 10054 21351 22598 34862 4028...,12
3,5,13176 47329 27966 23909 48370 13245 27360 6348...,22
4,6,40462,1


## 3.3. Retain baskets with more than one product

In [47]:
# keep orders with more than one product only
prior_orders = prior_orders.loc[prior_orders["num_products"] > 1, :]
print(prior_orders.shape)

(3029493, 3)


----

# 4. Train dataset orders

### We have made our own training and test data
### This dataset will eventually be appended to the larger prior orders

In [48]:
# training orders
train = pd.read_csv(data_dir + "order_products__train.csv")
print(train.shape)

(1384617, 4)


In [49]:
display(train.head())

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


## 4.1. Remove in-frequent products

In [50]:
# retain only frequently sold products
train = train.loc[train["product_id"].isin(list(prod_freq.index)), :]
print(train.shape)

(1298865, 4)


## 4.2. Aggregate to order-level

In [51]:
train["product_id"] = train["product_id"].astype(str)
train_orders = train.groupby(["order_id"]).agg(f)
train_orders.shape

(130524, 2)

## 4.3. Retain baskets with more than one product

In [52]:
# reset columns - was a pain to figure this out
train_orders.columns = train_orders.columns.droplevel(1)
train_orders.reset_index(inplace = True)
train_orders.rename(columns = {"add_to_cart_order" : "num_products"}, inplace = True)
train_orders = train_orders.loc[train_orders["num_products"] > 1, :]
print(train_orders.shape)

(123062, 3)


In [53]:
display(train_orders.head())

,order_id,product_id,num_products
0,1,11109 10246 49683 43633 13176 47209 22035,7
1,36,39612 19660 49235 43086 46620 34497 48679 46979,8
2,38,11913 18159 4461 21616 23622 32433 28842 42625...,9
3,96,20574 30391 40706 25610 27966 24489 39275,7
4,98,8859 19731 43654 13176 4357 37664 34065 35951 ...,49


----

# 5. Combine training and prior orders

In [54]:
# =========================================================
# add identifier column to both data sets to append
# =========================================================

# orders-level data
prior_orders["eval"] = "prior"
train_orders["eval"] = "train"

# order-product level data
prior["eval"] = "prior"
train["eval"] = "train"

In [55]:
display(prior_orders.head())

,order_id,product_id,num_products,eval
0,2,33120 28985 9327 45918 30035 17794 40141 1819,8,prior
1,3,33754 24838 17704 21903 17668 46667 17461 32665,8,prior
2,4,46842 39758 27761 10054 21351 22598 34862 4028...,12,prior
3,5,13176 47329 27966 23909 48370 13245 27360 6348...,22,prior
5,7,34050 46802,2,prior


In [56]:
display(train_orders.head())

,order_id,product_id,num_products,eval
0,1,11109 10246 49683 43633 13176 47209 22035,7,train
1,36,39612 19660 49235 43086 46620 34497 48679 46979,8,train
2,38,11913 18159 4461 21616 23622 32433 28842 42625...,9,train
3,96,20574 30391 40706 25610 27966 24489 39275,7,train
4,98,8859 19731 43654 13176 4357 37664 34065 35951 ...,49,train


In [57]:
display(prior.head())

,order_id,product_id,add_to_cart_order,reordered,eval
0,2,33120,1,1,prior
1,2,28985,2,1,prior
2,2,9327,3,0,prior
3,2,45918,4,1,prior
4,2,30035,5,0,prior


In [58]:
display(train.head())

,order_id,product_id,add_to_cart_order,reordered,eval
1,1,11109,2,1,train
2,1,10246,3,0,train
3,1,49683,4,0,train
4,1,43633,5,1,train
5,1,13176,6,0,train


## 5.1. Append order-level data

In [59]:
# append and save
all_orders_wide = prior_orders.append(train_orders)
print(all_orders_wide.shape)

(3152555, 4)


In [60]:
display(all_orders_wide.head())

,order_id,product_id,num_products,eval
0,2,33120 28985 9327 45918 30035 17794 40141 1819,8,prior
1,3,33754 24838 17704 21903 17668 46667 17461 32665,8,prior
2,4,46842 39758 27761 10054 21351 22598 34862 4028...,12,prior
3,5,13176 47329 27966 23909 48370 13245 27360 6348...,22,prior
5,7,34050 46802,2,prior


In [61]:
# reset index
all_orders_wide.reset_index(drop = True, inplace = True)

In [62]:
display(all_orders_wide.head())

,order_id,product_id,num_products,eval
0,2,33120 28985 9327 45918 30035 17794 40141 1819,8,prior
1,3,33754 24838 17704 21903 17668 46667 17461 32665,8,prior
2,4,46842 39758 27761 10054 21351 22598 34862 4028...,12,prior
3,5,13176 47329 27966 23909 48370 13245 27360 6348...,22,prior
4,7,34050 46802,2,prior


## 5.2. Save to disk

In [63]:
# save
all_orders_wide.to_csv(process_dir + "all-orders-wide-v1.csv", index = False)

## 5.3. Append order-product level data

In [64]:
# append and save
all_orders_long = prior.append(train)
print(all_orders_long.shape)

(32019330, 5)


In [65]:
display(all_orders_long.head())

,order_id,product_id,add_to_cart_order,reordered,eval
0,2,33120,1,1,prior
1,2,28985,2,1,prior
2,2,9327,3,0,prior
3,2,45918,4,1,prior
4,2,30035,5,0,prior


In [66]:
# reset index
all_orders_long.reset_index(drop = True, inplace = True)

## 5.2. Save to disk

In [44]:
# save
all_orders_long.to_csv(process_dir + "all-orders-long-v1.csv", index = False)

--------